# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096987


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/33 [00:00<?, ?it/s]

Rendering models:   6%|▌         | 2/33 [00:04<01:05,  2.13s/it]

Rendering models:   9%|▉         | 3/33 [00:06<01:08,  2.28s/it]

Rendering models:  12%|█▏        | 4/33 [00:06<00:47,  1.63s/it]

Rendering models:  15%|█▌        | 5/33 [00:07<00:32,  1.18s/it]

...

Rendering models:  82%|████████▏ | 27/33 [00:08<00:00, 10.37it/s]

Rendering models:  88%|████████▊ | 29/33 [00:09<00:00, 10.78it/s]

Rendering models:  94%|█████████▍| 31/33 [00:09<00:00, 11.32it/s]

Rendering models: 100%|██████████| 33/33 [00:09<00:00, 11.54it/s]

not-logged-in-3b0b929f4475fd276561     0.000241
Alexander00                            0.000973
jnarayanbvg                            0.105414
ElisabethB                             0.000232
jnarayanbvg                            0.000494
awright5                               0.001233
Galactea                               0.038848
thiagoxxxx                             0.000249
framosuw                               0.000659
AaronWallage117                        0.129267
not-logged-in-f7e2260ed4a9517a3e4a     0.012794
not-logged-in-f7e2260ed4a9517a3e4a     0.001044
sn346808                               0.005371
not-logged-in-f7e2260ed4a9517a3e4a     0.000427
daniela.araya.monge                    0.001026
mami2019                               0.002149
Tenorclef                              0.000615
umtuck1                                0.000687
not-logged-in-be875f27f6119e1941de    22.210408
not-logged-in-be875f27f6119e1941de     0.003376
Hansson                                0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())